In [1]:
#import librarys

import requests
import pandas as pd
from datetime import timedelta, date

In [2]:
# Define daterange function

def daterange(start_date, end_date):
    """Function to create a list of dates between the start and end date"""
    for n in range(int ((end_date - start_date).days)):
        current_day = start_date + timedelta(n)
        formatted_date = current_day.strftime("%m-%d-%Y")
        yield formatted_date

In [3]:
# Import and create dataset

def pull_multi_index_dataframe(start_date, end_date):
    """ Uses dateframe to create range of dates to concatenate all data to single multi-index dataframe"""
    
    # Pull US data for each day and concatenate into a single dataframe
    df = pd.DataFrame()
    for single_date in daterange(start_date, end_date):
        date = single_date
        url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports_us/' + date + '.csv'
        raw_data = pd.read_csv(url, error_bad_lines=False)
        raw_data['Date'] = single_date
        df = pd.concat([df, raw_data])

    # Set multi-index for dataframe
    df.set_index(['Province_State','Date'], inplace=True)
    df.sort_index(inplace=True)

    # Remove irrelavent rows
    df.drop(index=["American Samoa","Diamond Princess","Recovered"], level=0, inplace=True)
    df.drop(["Last_Update", "Lat", "Long_", "Recovered", "Active", "FIPS", "Incident_Rate", "People_Hospitalized", "Mortality_Rate", "UID", "ISO3", "Testing_Rate", "Hospitalization_Rate"], axis=1, inplace=True)
    df = df[df.Country_Region == 'US']
    
    return df

# Run function
start_date = date(2020, 4, 12)
end_date = date(2020, 4, 23)
today = date.today()
day = timedelta(1)
end_date = today - day
data = pull_multi_index_dataframe(start_date, end_date)

print(data)

                          Country_Region  Confirmed  Deaths  People_Tested
Province_State Date                                                       
Alabama        04-12-2020             US       3563      93        21583.0
               04-13-2020             US       3734      99        29182.0
               04-14-2020             US       3953     114        33117.0
               04-15-2020             US       4075     118        34077.0
               04-16-2020             US       4345     133        36391.0
...                                  ...        ...     ...            ...
Wyoming        04-20-2020             US        317       2         7386.0
               04-21-2020             US        443       2         7621.0
               04-22-2020             US        447       6         7623.0
               04-23-2020             US        453       7         7567.0
               04-24-2020             US        473       7         8045.0

[728 rows x 4 columns]


In [4]:
# Create new column with new tests
def add_daily_confirmed(df):
    """Function that adds a 'Daily Confirmed' column to the inputted dataframe"""
    previous = 0
    for i, row in df.iterrows():
        df.loc[i, 'Daily_Confirmed'] = row['Confirmed'] - previous
        previous = row['Confirmed']
    return df   

data = add_daily_confirmed(data)

In [5]:
# Create new column with 14 day average of new cases
def fourteen_day_avg_new(df):
    """Add 'fourteen_day_avg' column"""
    df['14d_average_new'] = df['Daily_Confirmed'].rolling(window=5).mean()
    return df

data = fourteen_day_avg_new(data)

In [6]:
# Create new column with daily tests
def add_daily_tests(df):
    """Function that adds a 'Daily Tests' column to the inputted dataframe"""
    previous = 0
    for i, row in df.iterrows():
        df.loc[i, 'Daily_Tests'] = row['People_Tested'] - previous
        previous = row['People_Tested']
    return df   

data = add_daily_tests(data)

In [7]:
# Create new column defining positive tests as percent of total tests
def add_daily_tests_pos_perc(df):
    """Function that adds a column to define positive tests as percent of total tests"""
    for i, row in df.iterrows():
        if row['Daily_Tests'] == 0:
            pass
        else:
            df.loc[i, 'positive_test_percentage'] = row['Daily_Confirmed'] / row['Daily_Tests'] * 100
    return df   

data = add_daily_tests_pos_perc(data)

In [8]:
# Determine 14 day average of perecent positive tests
def fourteen_day_avg_pos_tests(df):
    """Add 'fourteen_day_avg' column"""
    df['14d_positive_test_percentage'] = df['positive_test_percentage'].rolling(window=5).mean()
    return df

data = fourteen_day_avg_pos_tests(data)

In [9]:
def percent_change_fourteen_day_avg_new(df):
    previous = 0
    for i, row in df.iterrows():
        if previous == 0:
            pass
        else:
            df.loc[i,"percent_change_fourteen_day_avg_new"] = ((row['14d_average_new'] - previous ) / previous) * 100
        previous = row['14d_average_new']
    return df

data = percent_change_fourteen_day_avg_new(data)

In [10]:
def percent_change_fourteen_day_avg_pos_tests(df):
    previous = 0
    for i, row in df.iterrows():
        if previous == 0:
            pass
        else:
            df.loc[i,"percent_change_14d_positive_test_percentage"] = ((row['14d_positive_test_percentage'] - previous ) / previous) * 100
        previous = row['14d_positive_test_percentage']
    return df

data = percent_change_fourteen_day_avg_pos_tests(data)

In [11]:
data.to_excel("output.xlsx")